Single Race Exploratory Data Analysis -

The purpose of this notebook is to explore and wrangle data from the Fast-F1 API for a single race. This serves as a starting point to establish an organized procedure for data acquisition, preparation, and early exploration before scaling to a multi-race analysis. Descriptive statistics and preliminary visualizations will be generated to provide an initial understanding of the data, identify patterns or anomalies, and inform subsequent feature engineering and modeling decisions.

The code below adds the parent directory to Python’s module search path and configures logging to suppress all FastF1 logs below the warning level. This will enable subsequent code blocks that use imports to work seamlessly and keep my resulting code compilations clean and easy to read.

In [23]:
import sys
import os
import logging

root = os.path.abspath("..")
sys.path.append(root)

logging.getLogger('fastf1').setLevel(logging.WARNING)

In this section, I import Python libraries for data visualization, numerical analysis, and working with Pandas dataframes that the FastF1 API is primarily structured with. I also import custom functions and modules for preprocessing F1 data and constants. To support full visibility into the datasets without truncation, I configure Pandas display options to show all rows and columns.

In [24]:
from src.data import f1_data
from src.utils import f1_constants, f1_pandas_helpers
from src.preprocessing import telemetry_cleaning

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', None)  # reset_option to compact view
pd.set_option('display.max_columns', None)


The following code initializes a single F1 race session by defining parameters such as year, location, and session type. These values are passed into the custom F1Session class (from f1_data.py), which creates a session object built on top of Fast-F1. This object provides access to race data as well as custom functions I’ve implemented.

The session parameters were chosen to best match Tier 1 control qualities:

- Weather: Abu Dhabi (dry conditions)
- Max Speed: C5 Ultra Soft Tires
- Minimize Outliers: Qualifying Sessions (Q3 & Q2)
- Traffic: Avoid tow laps to avoid slipstream bias

In [25]:
year = 2024
grand_prix = f1_constants.F1Constants.LOCATIONS["Abu Dhabi"]
session_type = f1_constants.F1Constants.SESSIONS["Q"]

session = f1_data.F1Session(year, grand_prix, session_type)

All drivers who participated in the specific location's Q2 and Q3 sessions will be analyzed and assigned variables to be identified by their three-letter name code. Q3 data will be used for each driver whenever available; otherwise, Q2 data will be selected.

In [26]:
norris = f1_constants.F1Constants.DRIVERS["Lando Norris"]
piastri = f1_constants.F1Constants.DRIVERS["Oscar Piastri"]
verstappen = f1_constants.F1Constants.DRIVERS["Max Verstappen"]
perez = f1_constants.F1Constants.DRIVERS["Sergio Perez"]
sainz = f1_constants.F1Constants.DRIVERS["Carlos Sainz"]
leclerc = f1_constants.F1Constants.DRIVERS["Charles Leclerc"]
bottas = f1_constants.F1Constants.DRIVERS["Valtteri Bottas"]
alonso = f1_constants.F1Constants.DRIVERS["Fernando Alonso"]
stroll = f1_constants.F1Constants.DRIVERS["Lance Stroll"]
gasly = f1_constants.F1Constants.DRIVERS["Pierre Gasly"]
hulkenberg = f1_constants.F1Constants.DRIVERS["Nico Hulkenberg"]
magnussen = f1_constants.F1Constants.DRIVERS["Kevin Magnussen"]
lawson = f1_constants.F1Constants.DRIVERS["Liam Lawson"]
tsunoda = f1_constants.F1Constants.DRIVERS["Yuki Tsunoda"]
russell = f1_constants.F1Constants.DRIVERS["George Russell"]

For each driver, their fastest qualifying lap is identified by filtering timestamps for each sector. The resulting sector datasets are cleaned and used to derived additoinal features to identify deeper insights into drivers' response to track dynamics.

In [ ]:
# Q3 data for Lando Norris, McLaren
q1, q2, q3 = session.get_laps(norris).split_qualifying_sessions()
norris_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:16:59.496000"
sector_1_end = "0 days 01:17:16.260000"
sector_2_end = "0 days 01:17:52.036000"
sector_3_end = "0 days 01:18:21.897000"

# norris_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(norris_q3_telemetry, start=sector_1_start, end=sector_1_end)
# norris_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(norris_sector1_telemetry, norris)
# norris_sector1_telemetry_cleaned

# norris_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(norris_q3_telemetry, start=sector_1_end, end=sector_2_end)
# norris_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(norris_sector2_telemetry, norris)
# norris_sector2_telemetry_cleaned

norris_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(norris_q3_telemetry, start=sector_2_end, end=sector_3_end)
norris_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(norris_sector3_telemetry, norris)
norris_sector3_telemetry_cleaned

In [6]:
# Q3 data for Oscar Piastri, McLaren
q1, q2, q3 = session.get_laps(piastri).split_qualifying_sessions()
piastri_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:16:27.506000"
sector_1_end = "0 days 01:16:44.333000"
sector_2_end = "0 days 01:17:20.186000"
sector_3_end = "0 days 01:17:50.110000"

# piastri_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(piastri_q3_telemetry, start=sector_1_start, end=sector_1_end)
# piastri_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(piastri_sector1_telemetry, piastri)
# piastri_sector1_telemetry_cleaned

# piastri_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(piastri_q3_telemetry, start=sector_1_end, end=sector_2_end)
# piastri_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(piastri_sector2_telemetry, piastri)
# piastri_sector2_telemetry_cleaned

piastri_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(piastri_q3_telemetry, start=sector_2_end, end=sector_3_end)
piastri_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(piastri_sector3_telemetry, piastri)
piastri_sector3_telemetry_cleaned

,DriverCode,SessionTime (s),SectorTime (s),RPM,Speed (m/s),nGear,Throttle (%),BrakesApplied,Distance (m),X (1/10 m),Y (1/10 m),Z (1/10 m)
8159,PIA,01:17:20.195,00:17:43.940,11072.939722,87.805554,8,40.999936,0,24488.931434,4469.000000,-5127.000000,-230.000000
8160,PIA,01:17:20.277,00:17:44.022,10895.000000,86.666667,8,0.000000,1,24495.916944,4518.342601,-5146.361327,-229.949366
8161,PIA,01:17:20.434,00:17:44.179,10455.400179,81.651391,8,0.000000,1,24508.637455,4580.000000,-5171.000000,-230.000000
8162,PIA,01:17:20.477,00:17:44.222,10335.000000,80.277778,8,0.000000,1,24511.972500,4606.842154,-5181.564809,-229.847388
8163,PIA,01:17:20.633,00:17:44.378,10959.000000,74.319444,8,0.000000,1,24523.551990,4728.000000,-5229.000000,-229.000000
8164,PIA,01:17:20.637,00:17:44.382,10975.000000,74.166667,8,0.000000,1,24523.839167,4730.986183,-5230.169971,-228.985219
8165,PIA,01:17:20.873,00:17:44.618,10992.044444,69.614198,8,0.000000,1,24539.985819,4888.000000,-5291.000000,-229.000000
8166,PIA,01:17:20.997,00:17:44.742,11001.000000,67.222222,8,0.000000,1,24548.039167,4963.105829,-5317.491724,-229.688660
8167,PIA,01:17:21.054,00:17:44.799,11164.910316,65.961858,8,0.000000,1,24551.672515,4999.000000,-5327.000000,-230.000000
8168,PIA,01:17:21.198,00:17:44.943,11579.000000,62.777778,6,0.000000,1,24560.657500,5097.931768,-5341.042169,-230.497844


In [7]:
# Q3 data for Max Verstappen, Red Bull
q1, q2, q3 = session.get_laps(verstappen).split_qualifying_sessions()
verstappen_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:08:27.031000"
sector_1_end = "0 days 01:08:44.036000"
sector_2_end = "0 days 01:09:19.766000"
sector_3_end = "0 days 01:09:49.959000"

# verstappen_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(verstappen_q3_telemetry, start=sector_1_start, end=sector_1_end)
# verstappen_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(verstappen_sector1_telemetry, verstappen)
# verstappen_sector1_telemetry_cleaned

# verstappen_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(verstappen_q3_telemetry, start=sector_1_end, end=sector_2_end)
# verstappen_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(verstappen_sector2_telemetry, verstappen)
# verstappen_sector2_telemetry_cleaned

verstappen_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(verstappen_q3_telemetry, start=sector_2_end, end=sector_3_end)
verstappen_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(verstappen_sector3_telemetry, verstappen)
verstappen_sector3_telemetry_cleaned


,DriverCode,SessionTime (s),SectorTime (s),RPM,Speed (m/s),nGear,Throttle (%),BrakesApplied,Distance (m),X (1/10 m),Y (1/10 m),Z (1/10 m)
9139,VER,01:09:19.874,00:20:15.171,11422.871047,89.847671,8,29.032320,0,8822.800856,4434.000000,-5114.000000,-230.000000
9140,VER,01:09:19.955,00:20:15.252,11386.000000,89.444444,8,0.000000,1,8829.808333,4511.135071,-5143.983356,-229.672334
9141,VER,01:09:20.174,00:20:15.471,10698.203728,81.840284,8,0.000000,1,8847.284928,4726.000000,-5228.000000,-229.000000
9142,VER,01:09:20.275,00:20:15.572,10381.000000,78.333333,8,0.000000,1,8854.875000,4782.499755,-5250.769217,-228.851023
9143,VER,01:09:20.434,00:20:15.731,10351.585036,73.695839,8,0.000000,1,8866.443058,4849.000000,-5277.000000,-229.000000
9144,VER,01:09:20.475,00:20:15.772,10344.000000,72.500000,8,0.000000,1,8869.375000,4872.320023,-5285.526869,-229.160848
9145,VER,01:09:20.634,00:20:15.931,10633.180901,70.015628,8,0.000000,1,8880.507433,4976.000000,-5321.000000,-230.000000
9146,VER,01:09:20.635,00:20:15.932,10635.000000,70.000000,7,0.000000,1,8880.575000,4976.608033,-5321.190491,-230.004011
9147,VER,01:09:20.875,00:20:16.172,11236.000000,63.611111,5,0.000000,1,8895.841667,5101.739971,-5350.009412,-230.417191
9148,VER,01:09:20.994,00:20:16.291,11066.635083,61.416554,5,0.000000,1,8902.976243,5182.000000,-5345.000000,-231.000000


In [8]:
# Q3 data for Sergio Perez, Red Bull
q1, q2, q3 = session.get_laps(perez).split_qualifying_sessions()
perez_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:14:50.040000"
sector_1_end = "0 days 01:15:06.945000"
sector_2_end = "0 days 01:15:42.890000"
sector_3_end = "0 days 01:16:13.127000"

# perez_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(perez_q3_telemetry, start=sector_1_start, end=sector_1_end)
# perez_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(perez_sector1_telemetry, perez)
# perez_sector1_telemetry_cleaned

# perez_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(perez_q3_telemetry, start=sector_1_end, end=sector_2_end)
# perez_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(perez_sector2_telemetry, perez)
# perez_sector2_telemetry_cleaned

perez_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(perez_q3_telemetry, start=sector_2_end, end=sector_3_end)
perez_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(perez_sector3_telemetry, perez)
perez_sector3_telemetry_cleaned

,DriverCode,SessionTime (s),SectorTime (s),RPM,Speed (m/s),nGear,Throttle (%),BrakesApplied,Distance (m),X (1/10 m),Y (1/10 m),Z (1/10 m)
7372,PER,01:15:42.933,00:16:01.230,11414.437311,89.114825,8,40.111413,0,24513.357526,4501.000000,-5140.000000,-230.000000
7373,PER,01:15:43.077,00:16:01.374,11329.000000,87.777778,8,0.000000,1,24525.578333,4612.215407,-5183.090058,-229.497804
7374,PER,01:15:43.317,00:16:01.614,11028.000000,77.500000,8,0.000000,1,24544.178333,4721.729450,-5225.930630,-229.023316
7375,PER,01:15:43.334,00:16:01.631,10994.765125,77.051391,8,0.000000,1,24545.436085,4732.000000,-5230.000000,-229.000000
7376,PER,01:15:43.517,00:16:01.814,10637.000000,72.222222,7,0.000000,1,24558.622778,4860.207552,-5280.800769,-228.955249
7377,PER,01:15:43.533,00:16:01.830,10646.100000,71.972222,7,0.000000,1,24559.757122,4871.000000,-5285.000000,-229.000000
7378,PER,01:15:43.677,00:16:01.974,10728.000000,69.722222,7,0.000000,1,24569.778333,4961.688303,-5317.586467,-229.788617
7379,PER,01:15:43.714,00:16:02.011,10838.999808,68.865742,7,0.000000,1,24572.248845,4988.000000,-5324.000000,-230.000000
7380,PER,01:15:43.917,00:16:02.214,11448.000000,64.166667,7,0.000000,1,24585.178333,5159.937946,-5343.752580,-230.858148
7381,PER,01:15:43.954,00:16:02.251,11445.687504,63.717015,7,0.000000,1,24587.502640,5186.000000,-5345.000000,-231.000000


In [9]:
# Q3 data for Carlos Sainz, Ferrari
q1, q2, q3 = session.get_laps(sainz).split_qualifying_sessions()
sainz_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:16:53.219000"
sector_1_end = "0 days 01:17:10.014000"
sector_2_end = "0 days 01:17:45.801000"
sector_3_end = "0 days 01:18:15.858000"

# sainz_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(sainz_q3_telemetry, start=sector_1_start, end=sector_1_end)
# sainz_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(sainz_sector1_telemetry, sainz)
# sainz_sector1_telemetry_cleaned

# sainz_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(sainz_q3_telemetry, start=sector_1_end, end=sector_2_end)
# sainz_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(sainz_sector2_telemetry, sainz)
# sainz_sector2_telemetry_cleaned

sainz_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(sainz_q3_telemetry, start=sector_2_end, end=sector_3_end)
sainz_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(sainz_sector3_telemetry, sainz)
sainz_sector3_telemetry_cleaned

,DriverCode,SessionTime (s),SectorTime (s),RPM,Speed (m/s),nGear,Throttle (%),BrakesApplied,Distance (m),X (1/10 m),Y (1/10 m),Z (1/10 m)
7969,SAI,01:17:45.815,00:17:15.181,11403.605068,90.045834,8,13.060029,0,24539.332360,4339.000000,-5077.000000,-230.000000
7970,SAI,01:17:45.837,00:17:15.203,11392.000000,90.000000,8,8.000000,1,24541.243333,4363.610797,-5086.554693,-229.951838
7971,SAI,01:17:46.035,00:17:15.401,10259.439268,83.949996,8,0.079995,1,24557.860336,4602.000000,-5179.000000,-230.000000
7972,SAI,01:17:46.037,00:17:15.403,10248.000000,83.888889,8,0.000000,1,24558.021111,4603.712616,-5179.666685,-229.997587
7973,SAI,01:17:46.237,00:17:15.603,9595.000000,76.388889,8,0.000000,1,24573.298889,4686.202854,-5212.268178,-229.273409
7974,SAI,01:17:46.315,00:17:15.681,9961.043458,73.990075,8,0.000000,1,24578.822426,4713.000000,-5223.000000,-229.000000
7975,SAI,01:17:46.517,00:17:15.883,10909.000000,67.777778,8,0.000000,1,24592.276667,4833.233344,-5270.510924,-228.749108
7976,SAI,01:17:46.635,00:17:16.001,11272.956645,66.138887,8,0.000000,1,24599.897976,4909.000000,-5299.000000,-229.000000
7977,SAI,01:17:46.834,00:17:16.200,11886.746678,63.375001,8,0.000000,1,24612.359888,5025.000000,-5333.000000,-230.000000
7978,SAI,01:17:46.837,00:17:16.203,11896.000000,63.333333,7,0.000000,1,24612.543333,5027.066644,-5333.344195,-230.013543


In [10]:
# Q2 data for Charles Leclerc, Ferrari
q1, q2, q3 = session.get_laps(leclerc).split_qualifying_sessions()
leclerc_q2_telemetry = session.get_telemetry(q2)

sector_1_start = "0 days 00:57:03.473000"
sector_1_end = "0 days 00:57:20.332000"
sector_2_end = "0 days 00:57:56.296000"
sector_3_end = "0 days 00:58:26.338000"

# leclerc_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(leclerc_q2_telemetry, start=sector_1_start, end=sector_1_end)
# leclerc_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(leclerc_sector1_telemetry, leclerc)
# leclerc_sector1_telemetry_cleaned

# leclerc_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(leclerc_q2_telemetry, start=sector_1_end, end=sector_2_end)
# leclerc_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(leclerc_sector2_telemetry, leclerc)
# leclerc_sector2_telemetry_cleaned

leclerc_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(leclerc_q2_telemetry, start=sector_2_end, end=sector_3_end)
leclerc_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(leclerc_sector3_telemetry, leclerc)
leclerc_sector3_telemetry_cleaned

,DriverCode,SessionTime (s),SectorTime (s),RPM,Speed (m/s),nGear,Throttle (%),BrakesApplied,Distance (m),X (1/10 m),Y (1/10 m),Z (1/10 m)
10357,LEC,00:57:56.313,00:22:57.462,10994.333333,84.074074,8,35.000000,0,24485.250805,4434.000000,-5113.000000,-230.000000
10358,LEC,00:57:56.433,00:22:57.582,10779.000000,81.388889,8,3.000000,1,24494.582500,4552.167108,-5159.196784,-229.514313
10359,LEC,00:57:56.713,00:22:57.862,10523.000000,72.222222,8,2.000000,1,24514.804722,4759.434144,-5241.776188,-229.001597
10360,LEC,00:57:56.714,00:22:57.863,10522.840806,72.200112,8,2.000000,1,24514.874293,4760.000000,-5242.000000,-229.000000
10361,LEC,00:57:56.914,00:22:58.063,10491.000000,67.777778,8,2.000000,1,24528.428056,4848.553327,-5276.563254,-228.693326
10362,LEC,00:57:57.074,00:22:58.223,10833.666667,65.370370,8,1.666667,1,24538.544716,4913.000000,-5300.000000,-229.000000
10363,LEC,00:57:57.374,00:22:58.523,11476.166667,60.856481,8,1.041667,1,24556.315184,5085.000000,-5343.000000,-231.000000
10364,LEC,00:57:57.394,00:22:58.543,11519.000000,60.555556,5,1.000000,1,24557.494722,5100.350536,-5343.831503,-231.034311
10365,LEC,00:57:57.554,00:22:58.703,11225.285714,59.444444,5,4.428571,1,24566.931511,5230.000000,-5340.000000,-231.000000
10366,LEC,00:57:57.674,00:22:58.823,11005.000000,58.611111,5,7.000000,1,24573.905833,5301.661656,-5330.356077,-231.690289


In [28]:
# Q3 data for Valtteri Bottas, Alfa Romeo
q1, q2, q3 = session.get_laps(bottas).split_qualifying_sessions()
bottas_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:15:46.639000"
sector_1_end = "0 days 01:16:03.650000"
sector_2_end = "0 days 01:16:39.552000"
sector_3_end = "0 days 01:17:09.702000"

# bottas_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(bottas_q3_telemetry, start=sector_1_start, end=sector_1_end)
# bottas_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(bottas_sector1_telemetry, bottas)
# bottas_sector1_telemetry_cleaned

# bottas_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(bottas_q3_telemetry, start=sector_1_end, end=sector_2_end)
# bottas_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(bottas_sector2_telemetry, bottas)
# bottas_sector2_telemetry_cleaned

bottas_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(bottas_q3_telemetry, start=sector_2_end, end=sector_3_end)
bottas_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(bottas_sector3_telemetry, bottas)
bottas_sector3_telemetry_cleaned

,DriverCode,SessionTime (s),SectorTime (s),RPM,Speed (m/s),nGear,Throttle (%),BrakesApplied,Distance (m),X (1/10 m),Y (1/10 m),Z (1/10 m)
7153,BOT,01:16:39.597,00:15:27.992,11403.000000,89.722222,8,100.000000,1,8838.038056,4382.438363,-5093.401159,-230.821314
7154,BOT,01:16:39.834,00:15:28.229,10628.010209,81.822224,8,40.750016,1,8856.767338,4587.000000,-5173.000000,-230.000000
7155,BOT,01:16:39.997,00:15:28.392,10095.000000,76.388889,8,0.000000,1,8868.593611,4701.742126,-5218.646108,-229.059312
7156,BOT,01:16:40.014,00:15:28.409,10065.371540,75.899803,8,0.000000,1,8869.800127,4718.000000,-5225.000000,-229.000000
7157,BOT,01:16:40.195,00:15:28.590,9749.914063,70.692457,8,0.000000,1,8882.315350,4929.000000,-5306.000000,-229.000000
7158,BOT,01:16:40.277,00:15:28.672,9607.000000,68.333333,8,0.000000,1,8887.726944,4982.198106,-5325.564266,-229.205183
7159,BOT,01:16:40.495,00:15:28.890,9855.277632,64.128089,8,0.000000,1,8901.255113,5024.000000,-5333.000000,-230.000000
7160,BOT,01:16:40.637,00:15:29.032,10017.000000,61.388889,6,0.000000,1,8909.826944,5089.582398,-5342.716869,-230.413274
7161,BOT,01:16:40.775,00:15:29.170,10656.111907,60.190973,6,0.000000,1,8918.146008,5179.000000,-5345.000000,-231.000000
7162,BOT,01:16:40.797,00:15:29.192,10758.000000,60.000000,5,0.000000,1,8919.426944,5194.533742,-5343.488729,-231.135766


In [12]:
# Q3 data for Fernando Alonso, Aston Martin
q1, q2, q3 = session.get_laps(alonso).split_qualifying_sessions()
alonso_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:16:45.371000"
sector_1_end = "0 days 01:17:02.191000"
sector_2_end = "0 days 01:17:38.197000"
sector_3_end = "0 days 01:18:08.377000"

# alonso_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(alonso_q3_telemetry, start=sector_1_start, end=sector_1_end)
# alonso_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(alonso_sector1_telemetry, alonso)
# alonso_sector1_telemetry_cleaned

# alonso_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(alonso_q3_telemetry, start=sector_1_end, end=sector_2_end)
# alonso_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(alonso_sector2_telemetry, alonso)
# alonso_sector2_telemetry_cleaned

alonso_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(alonso_q3_telemetry, start=sector_2_end, end=sector_3_end)
alonso_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(alonso_sector3_telemetry, alonso)
alonso_sector3_telemetry_cleaned

,DriverCode,SessionTime (s),SectorTime (s),RPM,Speed (m/s),nGear,Throttle (%),BrakesApplied,Distance (m),X (1/10 m),Y (1/10 m),Z (1/10 m)
8058,ALO,01:17:38.415,00:17:28.949,11263.912646,82.534726,8,45.787526,0,24512.859471,4642.000000,-5195.000000,-230.000000
8059,ALO,01:17:38.637,00:17:29.171,11010.000000,76.111111,8,0.000000,1,24528.825556,4823.405274,-5264.804026,-229.903323
8060,ALO,01:17:38.774,00:17:29.308,10864.627846,72.939816,8,0.000000,1,24538.217757,4903.000000,-5297.000000,-230.000000
8061,ALO,01:17:38.997,00:17:29.531,10628.000000,67.777778,7,0.000000,1,24553.225556,4980.974317,-5334.587965,-230.257413
8062,ALO,01:17:39.237,00:17:29.771,11159.000000,67.777778,6,0.000000,1,24569.492222,5092.395603,-5346.682615,-230.921247
8063,ALO,01:17:39.255,00:17:29.789,11167.549939,67.170639,6,0.000000,1,24570.629074,5107.000000,-5346.000000,-231.000000
8064,ALO,01:17:39.454,00:17:29.988,11262.074970,60.458335,6,0.000000,1,24582.266242,5292.000000,-5328.000000,-232.000000
8065,ALO,01:17:39.517,00:17:30.051,11292.000000,58.333333,5,0.000000,1,24585.825556,5325.685528,-5322.530977,-232.229396
8066,ALO,01:17:39.714,00:17:30.248,10799.500160,56.574405,5,0.000000,1,24596.902268,5370.000000,-5303.000000,-233.000000
8067,ALO,01:17:39.797,00:17:30.331,10592.000000,55.833333,5,0.000000,1,24601.458889,5401.914418,-5287.656220,-233.723193


In [13]:
# Q2 data for Lance Stroll, Aston Martin
q1, q2, q3 = session.get_laps(stroll).split_qualifying_sessions()
stroll_q2_telemetry = session.get_telemetry(q2)

sector_1_start = "0 days 00:56:38.573000"
sector_1_end = "0 days 00:56:55.483000"
sector_2_end = "0 days 00:57:31.468000"
sector_3_end = "0 days 00:58:02.147000"

# stroll_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(stroll_q2_telemetry, start=sector_1_start, end=sector_1_end)
# stroll_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(stroll_sector1_telemetry, stroll)
# stroll_sector1_telemetry_cleaned

# stroll_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(stroll_q2_telemetry, start=sector_1_end, end=sector_2_end)
# stroll_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(stroll_sector2_telemetry, stroll)
# stroll_sector2_telemetry_cleaned

# stroll_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(stroll_q2_telemetry, start=sector_2_end, end=sector_3_end)
# stroll_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(stroll_sector3_telemetry, stroll)
# stroll_sector3_telemetry_cleaned

In [14]:
# Q3 data for Pierre Gasly, Alpine
q1, q2, q3 = session.get_laps(gasly).split_qualifying_sessions()
gasly_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:16:18.040000"
sector_1_end = "0 days 01:16:34.989000"
sector_2_end = "0 days 01:17:10.860000"
sector_3_end = "0 days 01:17:40.892000"

# gasly_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(gasly_q3_telemetry, start=sector_1_start, end=sector_1_end)
# gasly_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(gasly_sector1_telemetry, gasly)
# gasly_sector1_telemetry_cleaned

# gasly_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(gasly_q3_telemetry, start=sector_1_end, end=sector_2_end)
# gasly_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(gasly_sector2_telemetry, gasly)
# gasly_sector2_telemetry_cleaned

# gasly_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(gasly_q3_telemetry, start=sector_2_end, end=sector_3_end)
# gasly_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(gasly_sector3_telemetry, gasly)
# gasly_sector3_telemetry_cleaned

In [15]:
# Q3 data for Nico Hulkenberg, Haas
q1, q2, q3 = session.get_laps(hulkenberg).split_qualifying_sessions()
hulkenberg_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:16:03.696000"
sector_1_end = "0 days 01:16:20.651000"
sector_2_end = "0 days 01:16:56.369000"
sector_3_end = "0 days 01:17:26.431000"

# hulkenberg_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(hulkenberg_q3_telemetry, start=sector_1_start, end=sector_1_end)
# hulkenberg_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(hulkenberg_sector1_telemetry, hulkenberg)
# hulkenberg_sector1_telemetry_cleaned

# hulkenberg_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(hulkenberg_q3_telemetry, start=sector_1_end, end=sector_2_end)
# hulkenberg_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(hulkenberg_sector2_telemetry, hulkenberg)
# hulkenberg_sector2_telemetry_cleaned

# hulkenberg_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(hulkenberg_q3_telemetry, start=sector_2_end, end=sector_3_end)
# hulkenberg_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(hulkenberg_sector3_telemetry, hulkenberg)
# hulkenberg_sector3_telemetry_cleaned

In [16]:
# Q2 data for Kevin Magnussen, Haas
q1, q2, q3 = session.get_laps(magnussen).split_qualifying_sessions()
magnussen_q2_telemetry = session.get_telemetry(q2)

sector_1_start = "0 days 00:55:34.175000"
sector_1_end = "0 days 00:55:51.220000"
sector_2_end = "0 days 00:56:27.265000"
sector_3_end = "0 days 00:56:57.907000"

# magnussen_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(magnussen_q2_telemetry, start=sector_1_start, end=sector_1_end)
# magnussen_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(magnussen_sector1_telemetry, magnussen)
# magnussen_sector1_telemetry_cleaned

# magnussen_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(magnussen_q2_telemetry, start=sector_1_end, end=sector_2_end)
# magnussen_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(magnussen_sector2_telemetry, magnussen)
# magnussen_sector2_telemetry_cleaned

# magnussen_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(magnussen_q2_telemetry, start=sector_2_end, end=sector_3_end)
# magnussen_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(magnussen_sector3_telemetry, magnussen)
# magnussen_sector3_telemetry_cleaned

In [17]:
# Q2 data for Liam Lawson, Visa CashApp
q1, q2, q3 = session.get_laps(lawson).split_qualifying_sessions()
lawson_q2_telemetry = session.get_telemetry(q2)

sector_1_start = "0 days 00:57:38.695000"
sector_1_end = "0 days 00:57:55.568000"
sector_2_end = "0 days 00:58:31.626000"
sector_3_end = "0 days 00:59:01.907000"

# lawson_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(lawson_q2_telemetry, start=sector_1_start, end=sector_1_end)
# lawson_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(lawson_sector1_telemetry, lawson)
# lawson_sector1_telemetry_cleaned

# lawson_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(lawson_q2_telemetry, start=sector_1_end, end=sector_2_end)
# lawson_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(lawson_sector2_telemetry, lawson)
# lawson_sector2_telemetry_cleaned

# lawson_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(lawson_q2_telemetry, start=sector_2_end, end=sector_3_end)
# lawson_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(lawson_sector3_telemetry, lawson)
# lawson_sector3_telemetry_cleaned

In [18]:
# Q2 data for Yuki Tsunoda, Visa CashApp
q1, q2, q3 = session.get_laps(tsunoda).split_qualifying_sessions()
tsunoda_q2_telemetry = session.get_telemetry(q2)

sector_1_start = "0 days 00:57:26.587000"
sector_1_end = "0 days 00:57:43.456000"
sector_2_end = "0 days 00:58:19.489000"
sector_3_end = "0 days 00:58:49.753000"

# tsunoda_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(tsunoda_q2_telemetry, start=sector_1_start, end=sector_1_end)
# tsunoda_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(tsunoda_sector1_telemetry, tsunoda)
# tsunoda_sector1_telemetry_cleaned

# tsunoda_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(tsunoda_q2_telemetry, start=sector_1_end, end=sector_2_end)
# tsunoda_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(tsunoda_sector2_telemetry, tsunoda)
# tsunoda_sector2_telemetry_cleaned

# tsunoda_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(tsunoda_q2_telemetry, start=sector_2_end, end=sector_3_end)
# tsunoda_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(tsunoda_sector3_telemetry, tsunoda)
# tsunoda_sector3_telemetry_cleaned

In [19]:
# Q3 data for George Russell, Mercedes
q1, q2, q3 = session.get_laps(russell).split_qualifying_sessions()
russell_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:17:14.677000"
sector_1_end = "0 days 01:17:31.522000"
sector_2_end = "0 days 01:18:07.527000"
sector_3_end = "0 days 01:18:37.566000"

# russell_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(russell_q3_telemetry, start=sector_1_start, end=sector_1_end)
# russell_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(russell_sector1_telemetry, russell)
# russell_sector1_telemetry_cleaned

# russell_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(russell_q3_telemetry, start=sector_1_end, end=sector_2_end)
# russell_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(russell_sector2_telemetry, russell)
# russell_sector2_telemetry_cleaned

# russell_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(russell_q3_telemetry, start=sector_2_end, end=sector_3_end)
# russell_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(russell_sector3_telemetry, russell)
# russell_sector3_telemetry_cleaned

Sector 1 telemetry data for all drivers is concatanated into a single dataframe below.

In [20]:
# total_sector1_telemetry = pd.concat([
#     norris_sector1_telemetry_cleaned, 
#     piastri_sector1_telemetry_cleaned, 
#     verstappen_sector1_telemetry_cleaned, 
#     perez_sector1_telemetry_cleaned,
#     sainz_sector1_telemetry_cleaned,
#     leclerc_sector1_telemetry_cleaned,
#     bottas_sector1_telemetry_cleaned,
#     alonso_sector1_telemetry_cleaned,
#     stroll_sector1_telemetry_cleaned,
#     gasly_sector1_telemetry_cleaned,
#     hulkenberg_sector1_telemetry_cleaned,
#     magnussen_sector1_telemetry_cleaned,
#     lawson_sector1_telemetry_cleaned,
#     tsunoda_sector1_telemetry_cleaned,
#     russell_sector1_telemetry_cleaned
# ], axis=0)

# total_sector1_telemetry

Sector 2 telemetry data for all drivers is concatenated into a single dataframe below.

In [21]:
# total_sector2_telemetry = pd.concat([
#     norris_sector2_telemetry_cleaned, 
#     piastri_sector2_telemetry_cleaned, 
#     verstappen_sector2_telemetry_cleaned, 
#     perez_sector2_telemetry_cleaned,
#     sainz_sector2_telemetry_cleaned,
#     leclerc_sector2_telemetry_cleaned,
#     bottas_sector2_telemetry_cleaned,
#     alonso_sector2_telemetry_cleaned,
#     stroll_sector2_telemetry_cleaned,
#     gasly_sector2_telemetry_cleaned,
#     hulkenberg_sector2_telemetry_cleaned,
#     magnussen_sector2_telemetry_cleaned,
#     lawson_sector2_telemetry_cleaned,
#     tsunoda_sector2_telemetry_cleaned,
#     russell_sector2_telemetry_cleaned
# ], axis=0)

# total_sector2_telemetry

Sector 3 telemetry data for all drivers is concatanated into a single dataframe below.

In [22]:
# total_sector3_telemetry = pd.concat([
#     norris_sector3_telemetry_cleaned, 
#     piastri_sector3_telemetry_cleaned, 
#     verstappen_sector3_telemetry_cleaned, 
#     perez_sector3_telemetry_cleaned,
#     sainz_sector3_telemetry_cleaned,
#     leclerc_sector3_telemetry_cleaned,
#     bottas_sector3_telemetry_cleaned,
#     alonso_sector3_telemetry_cleaned,
#     stroll_sector3_telemetry_cleaned,
#     gasly_sector3_telemetry_cleaned,
#     hulkenberg_sector3_telemetry_cleaned,
#     magnussen_sector3_telemetry_cleaned,
#     lawson_sector3_telemetry_cleaned,
#     tsunoda_sector3_telemetry_cleaned,
#     russell_sector3_telemetry_cleaned
# ], axis=0)

# total_sector3_telemetry